In [126]:
import pandas as pd
import datetime
import xlsxwriter

try:
    df = pd.read_excel(open("rptDailyReportEnviro_naphtha.xls",'rb'), header = 1)
except:
    df = pd.read_excel(open("rptDailyReportEnviro.xls",'rb'), header = 1)

#remove unneeded rows and columns
data = df[:-1]
data.drop(['BOL_Label','BSW_Label'],inplace=True,axis=1)

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [127]:
#remover whitespace in consignee and product columns
data.dtypes
data["Text27"] = data["Text27"].map(str.strip)
data["Text98"]= data["Text98"].map(str.strip)

#sort by gross_date
data.sort_values('GROSS_DATE',inplace=True)

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [128]:
#remove naphtha loads that do no pass through the racks (i.e. offloaded at ChemOil or transactions at ChemOil)
data = data[data["Text98"] == "NAPHTHA PETROLEUM DISTILLATE"]
data = data[(data['Text27'] != 'FORELAND REFINING (NAPHTHA)') & (data['Text27'] !='TESORO (NAPHTHA)')] 
data = data[(data['NET_BARRELS']) > 0]

In [129]:
#C1.33 calcs

#sort by GROSS_DATE
dates = data["GROSS_DATE"].sort_values()

#sum gallons loaded per day
thruput = data.groupby(['GROSS_DATE'])['Text125'].sum()

#extract unique day values
data["dates"] = data["GROSS_DATE"].dt.strftime('%Y-%m-%d')
dates = data["dates"].unique()

#compliance status
status_thru = []
for thru in thruput:
    if thru <= 80000:
        stat = 'OK'
    else:
        stat = 'EXCEEDANCE'
    status_thru.append(stat)

In [130]:
#write to Excel

#setup workbook and worksheet
workbook = xlsxwriter.Workbook('%s to %s_Naphtha Loading_C1.33.xlsx' %(dates[0],dates[-1]))
bold = workbook.add_format({'bold':True})
num_fmt = workbook.add_format({'num_format':'###,###,###'})
worksheet1 = workbook.add_worksheet('Summary')

worksheet1.set_column('A:D',15)

row = 0
col = 0

#write C1.33 headers
worksheet1.write(row,col,"C1.33: Naphtha Racks 7 and 8 (D136) cannot exceed monthly throughput of 80,000 gallons.",bold)
row += 2
columns = ['Date','Throughput (gal)','Limit (gal)','Compliance']
for header in columns:
    worksheet1.write(row,col,header)
    col += 1
row += 1
col = 0

#write C1.33 data
for i in range(len(dates)):
    worksheet1.write(row,col,dates[i])
    col += 1
    worksheet1.write(row,col,thruput[i],num_fmt)
    col += 1
    worksheet1.write_number(row,col,80000,num_fmt)
    col += 1
    worksheet1.write(row,col,status_thru[i])
    row += 1
    col = 0
    
workbook.close()